# 📘 Big Data สัปดาห์ที่ 4: Distributed Processing & Apache Spark (Deep Dive)**รายวิชา:** ข้อมูลขนาดใหญ่ (Big Data) — 128-356  **คณะ:** คณะวิทยาการคอมพิวเตอร์ มหาวิทยาลัยสยาม  **สัปดาห์ที่:** 4  > 🎯 **เป้าหมายวันนี้:** เข้าใจแนวคิด Distributed Computing, สถาปัตยกรรมของ Apache Spark, Lazy Evaluation, DAG และ Shuffle พร้อมลงมือปฏิบัติจริงด้วย PySpark**ใช้ได้ทั้ง**: Google Colab (แนะนำ) / Local Jupyter  ---

# 1. 🖥️ ทำไมเครื่องเดียวถึงไม่พอ? (Why Single Machine Fails)## ปัญหาของการประมวลผลด้วยเครื่องเดียวเมื่อข้อมูลมีขนาดใหญ่ขึ้น เราจะพบข้อจำกัดหลายด้าน:| ข้อจำกัด | คำอธิบาย | ตัวอย่างปัญหา ||----------|----------|---------------|| **RAM** | หน่วยความจำมีจำกัด (เช่น 16 GB) | ลองโหลด CSV 50 GB ด้วย Pandas → MemoryError || **CPU** | จำนวน Core มีจำกัด (เช่น 4-8 Cores) | งาน groupBy บน 1 พันล้านแถว ใช้เวลาหลายชั่วโมง || **I/O** | Disk อ่าน/เขียนช้า | สแกนไฟล์ 100 GB จาก HDD ใช้เวลา ~15 นาที || **ไม่ขยายได้** | เพิ่ม RAM/CPU มีเพดาน | เครื่องแพงสุดในตลาดก็มีขีดจำกัด |### 💡 แนวคิดสำคัญ (Key Insight)> เมื่อ **ขนาดข้อมูล × ความซับซ้อน > ขีดความสามารถของเครื่องเดียว**  > → ต้องใช้ **Distributed Computing** (การประมวลผลแบบกระจาย)### เปรียบเทียบง่าย ๆ- **เครื่องเดียว** = คนเดียวยกอิฐ 10,000 ก้อน → เหนื่อย ช้า เสียเวลา- **Distributed** = คน 100 คน ช่วยกันยกคนละ 100 ก้อน → เสร็จเร็ว!

<div align="center">  <img src="images/single_vs_distributed.png" width="800" alt="Single Machine vs Distributed Computing: คนเดียวแบกของหนัก vs หลายคนช่วยกันแบก">  <br><i>Single Machine vs Distributed Computing: คนเดียวแบกของหนัก vs หลายคนช่วยกันแบก</i></div>

In [ ]:
# ตัวอย่าง: ลองดูข้อจำกัดของเครื่องเดียวimport psutil, osram_gb = psutil.virtual_memory().total / (1024**3)cpu_count = os.cpu_count()print(f"🖥️ เครื่องนี้มี RAM: {ram_gb:.1f} GB")print(f"🔧 จำนวน CPU Cores: {cpu_count}")print(f"\n💡 ถ้าข้อมูลใหญ่กว่า {ram_gb:.0f} GB → Pandas จะ crash!")print(f"💡 ถ้า query ซับซ้อน ใช้ได้แค่ {cpu_count} cores → ช้า!")

### ✏️ แบบฝึกหัดที่ 1: คำนวณขีดจำกัด**โจทย์:** บริษัทมีข้อมูล Log 500 GB/วัน ต้อง JOIN กับตาราง 50 GB และ GROUP BY  เครื่อง Server มี RAM 64 GB, 16 Cores**คำถาม** (เติมคำตอบในช่องว่าง):

In [ ]:
# แบบฝึกหัดที่ 1: เติมคำตอบ# 1) ข้อมูลรวมทั้งหมดกี่ GB?total_data_gb = ________  # เติมตัวเลข# 2) RAM เพียงพอหรือไม่? (True/False)ram_enough = ________  # True หรือ False# 3) ถ้าอ่าน disk ได้ 200 MB/s จะใช้เวลานานกี่วินาที (ประมาณ)?read_time_sec = ________  # เติมตัวเลข# 4) ควรใช้ Distributed Computing หรือไม่? เพราะอะไร?answer = "________"print(f"ข้อมูลรวม: {total_data_gb} GB")print(f"RAM เพียงพอ: {ram_enough}")print(f"เวลาอ่าน disk: {read_time_sec} วินาที ≈ {read_time_sec/60:.1f} นาที")print(f"คำตอบ: {answer}")

# 2. 🔥 Apache Spark คืออะไร?## นิยามApache Spark คือ **Distributed Computing Engine** (เครื่องยนต์ประมวลผลแบบกระจาย) สำหรับข้อมูลขนาดใหญ่> Spark เหมือน "สมองกลาง" ที่สั่งการให้เครื่องหลายร้อยตัวทำงานพร้อมกัน## คุณสมบัติหลัก| คุณสมบัติ | คำอธิบาย | เปรียบเทียบ ||-----------|----------|-------------|| **In-Memory Processing** | ประมวลผลใน RAM ไม่ต้องเขียน disk ทุกขั้นตอน | เร็วกว่า Hadoop MapReduce 10-100 เท่า || **DAG Execution** | สร้างแผนงานเป็น Graph ก่อนทำจริง | เหมือนวางแผนก่อนสร้างบ้าน || **Fault Tolerance** | ถ้าเครื่องพัง สามารถคำนวณใหม่ได้ | เหมือนมีสำเนาสูตรอาหาร || **Unified Engine** | รองรับ SQL, ML, Streaming, Graph | เครื่องมือเดียวทำได้หลายอย่าง |## ⚠️ Spark ไม่ใช่อะไร?- ❌ **ไม่ใช่ Database** — Spark ไม่เก็บข้อมูลถาวร- ❌ **ไม่ใช่ Storage System** — Spark อ่านข้อมูลจาก HDFS, S3, Parquet ฯลฯ- ✅ **Spark = Compute Layer** — เป็น "ชั้นประมวลผล" ที่อยู่ระหว่างข้อมูลกับผลลัพธ์### เปรียบเทียบ Spark กับเครื่องมืออื่น| เครื่องมือ | เหมาะกับ | ขนาดข้อมูล | ความเร็ว ||-----------|---------|-----------|---------|| **Pandas** | เครื่องเดียว, EDA | MB - ไม่เกิน RAM | เร็ว (ข้อมูลเล็ก) || **DuckDB** | SQL Analytics เครื่องเดียว | MB - GB | เร็วมาก || **Spark** | Distributed, Data Pipeline | GB - PB | เร็ว (ข้อมูลใหญ่) || **Hadoop MR** | Batch บน HDFS | TB - PB | ช้า (เขียน disk ทุกขั้น) |

# 3. 🏗️ สถาปัตยกรรม Spark เชิงลึก (Spark Architecture Deep Dive)## ส่วนประกอบหลัก 3 ส่วน```┌─────────────────────────────────────┐│         Cluster Manager             ││    (ผู้จัดสรรทรัพยากร: YARN/K8s)     │└──────────┬──────────────────────────┘           │ จัดสรร CPU/RAM    ┌──────┴──────┐    ▼             ▼┌────────┐   ┌────────────────────┐│ Driver │──▶│   Executors (N ตัว) ││ (สมอง) │   │   (แรงงาน)         │└────────┘   └────────────────────┘```### 🧠 Driver (สมอง) — หน้าที่:1. **สร้าง SparkSession** — เปิดประตูเข้าสู่ Spark2. **สร้าง Logical Plan** — แปลงโค้ดเป็นแผนงาน3. **Optimize** — ปรับแผนให้เร็วที่สุด (ผ่าน Catalyst Optimizer)4. **แบ่งงาน** — แบ่ง Job → Stages → Tasks5. **ติดตามผล** — ดูว่า Executor ทำเสร็จหรือยัง### ⚙️ Executor (แรงงาน) — หน้าที่:1. **รับ Task มาทำ** — ประมวลผลข้อมูลจริง ๆ2. **เก็บ Cache** — เก็บข้อมูลใน RAM สำหรับใช้ซ้ำ3. **Shuffle** — ส่งข้อมูลข้ามเครื่องเมื่อจำเป็น4. **รายงานผล** — ส่งผลกลับไปที่ Driver### 🔄 ลำดับการทำงาน: Job → Stage → Task → Partition| ระดับ | คำอธิบาย | เกิดจาก ||------|----------|--------|| **Job** | งานใหญ่ 1 ชิ้น | เกิดจาก Action (เช่น `.count()`) || **Stage** | ช่วงงานที่ทำได้โดยไม่ต้อง Shuffle | เกิดจากการตัดที่จุด Shuffle || **Task** | หน่วยงานย่อยที่สุด | 1 Task ประมวลผล 1 Partition || **Partition** | ชิ้นส่วนข้อมูล | ไฟล์ถูกแบ่งเป็น Partition อัตโนมัติ |> 💡 **เปรียบเทียบ:** Job = สร้างบ้าน, Stage = ขั้นตอน (เทฐาน/ก่อผนัง/มุงหลังคา), Task = ช่างแต่ละคน, Partition = วัสดุที่แบ่งให้ช่างแต่ละคน

<div align="center">  <img src="images/driver_internal.png" width="800" alt="Driver Internals Diagram">  <br><i>โครงสร้างภายในของ Driver: SparkSession, DAGScheduler, TaskScheduler</i></div>#### 🧠 เจาะลึก Driver Internals- **SparkSession**: จุดเริ่มต้นของการเขียนโค้ด (Entry Point)- **DAGScheduler**: แปลง Logical Plan เป็น Physical Plan และแบ่งงานเป็น Stages (Stage 1, Stage 2)- **TaskScheduler**: รับ Stage มาแตกเป็น Task ย่อยๆ แล้วส่งไปให้ Executor ที่ว่างอยู่- **SchedulerBackend**: สื่อสารกับ Cluster Manager เพื่อขอทรัพยากร (CPU/RAM)

<div align="center">  <img src="images/spark_chef_analogy.png" width="800" alt="Spark Architecture Analogy: Driver (Chef) สั่งงาน Executors (Cooks)">  <br><i>Spark Architecture Analogy: Driver (Chef) สั่งงาน Executors (Cooks)</i></div>

### 3.1 🧠 Deep Dive: SparkSession vs SparkContextหลายคนอาจสงสัยว่า `SparkSession` กับ `SparkContext` ต่างกันอย่างไร?#### 1. SparkContext (`sc`) — The Engine 🔧- เป็น **Entry Point** ดั้งเดิมของ Spark (ตั้งแต่เวอร์ชัน 1.x)- ทำหน้าที่เชื่อมต่อกับ Cluster Manager (YARN, K8s, Standalone)- จัดการ Executors, Memory, และ Job Scheduling- **เปรียบเทียบ:** เหมือน **"เครื่องยนต์"** ของรถยนต์ — ทำหน้าที่ขับเคลื่อนกลไกภายใน#### 2. SparkSession (`spark`) — The Dashboard 🚗- เริ่มใช้ใน Spark 2.0+- เป็น **Unified Entry Point** ที่รวมทุกอย่างไว้ในที่เดียว:  - `SparkContext` (Core)  - `SQLContext` (DataFrames/SQL)  - `HiveContext` (Hive tables)  - `StreamingContext` (Streaming)- **เปรียบเทียบ:** เหมือน **"คนขับ"** หรือ **"แผงหน้าปัด"** ที่เราใช้งานจริง (เราขับรถผ่านพวงมาลัย ไม่ได้ไปหมุนล้อเอง)> 💡 **Best Practice:** ในโค้ดปัจจุบัน **เราจะใช้ `SparkSession` เสมอ** (แต่เบื้องหลังมันก็เรียกใช้ `SparkContext` อยู่ดี)

## 🔬 Lab: เริ่มต้นใช้งาน Spark### Step 1: ติดตั้งและสร้าง SparkSession

In [ ]:
# ติดตั้ง PySpark (สำหรับ Colab)!pip -q install pysparkfrom pyspark.sql import SparkSessionimport os# สร้าง SparkSession — ประตูเข้าสู่ Spark# 💡 config("spark.ui.port", "4050") กำหนด port เพื่อเลี่ยง port 4040 ที่อาจชนกันspark = SparkSession.builder \    .appName("BigData-Week4-DeepDive") \    .config("spark.driver.memory", "2g") \    .config("spark.ui.port", "4050") \    .getOrCreate()# เข้าถึง SparkContext ที่ถูก wrap อยู่ข้างในsc = spark.sparkContextprint(f"✅ Spark Version: {spark.version}")print(f"📱 App Name: {sc.appName}")print(f"🖥️ Master: {sc.master}")

### 3.2 🛠️ แก้ปัญหา: เปิด Spark UI ใน Google Colab**ปัญหา:** ปกติ Spark UI จะอยู่ที่ `localhost:4040` แต่ Google Colab รันบนเครื่อง Server (Virtual Machine) ทำให้เราเข้าถึง `localhost` ไม่ได้โดยตรง**วิธีแก้:** เราต้องใช้ library `google.colab.output` เพื่อสร้าง Proxy ให้เราเปิดหน้าเว็บได้รัน cell นี้เพื่อเปิด Spark UI 👇

In [ ]:
# 🛠️ Code สำหรับเปิด Spark UI ใน Colabtry:    from google.colab import output        # ฟังก์ชันสำหรับเปิด Spark UI    def show_spark_ui(port=4050):        # ดึง URL ของ Spark UI ผ่าน Proxy ของ Colab        url = output.serve_kernel_port_as_iframe(port)        # หรือแสดงเป็น Link ให้คลิก (ถ้า iframe เล็กไป)        # output.serve_kernel_port_as_window(port)        print(f"🚀 Spark UI เปิดอยู่ที่: {url}")        print("💡 ถ้าไม่เห็นผลลัพธ์ ให้ลองคลิก link นี้แทน (อาจต้อง login Google):")        output.serve_kernel_port_as_window(port, path='/jobs/')    # เรียกใช้งาน (Default port ที่เราตั้งไว้คือ 4050)    show_spark_ui(4050)except ImportError:    print("⚠️ ไม่ได้รันบน Google Colab หรือไม่พบ library google.colab")    print(f"👉 ถ้าคุณรันบน Local Jupyter ให้เปิด: http://localhost:4050")

# 4. 🏢 Cluster Managers (ผู้จัดสรรทรัพยากร)Cluster Manager ทำหน้าที่จัดสรร CPU และ RAM ให้ Spark ใช้งาน เปรียบได้กับ "ผู้จัดการอาคาร" ที่จัดห้องให้ผู้เช่า## เปรียบเทียบ Cluster Manager 3 แบบ| | **Standalone** | **YARN** | **Kubernetes** ||---|---|---|---|| **ความหมาย** | Cluster Manager ของ Spark เอง | Hadoop ecosystem | Container-based || **ติดตั้ง** | ง่ายที่สุด | ต้องมี Hadoop | ต้องมี K8s cluster || **เหมาะกับ** | ทดลอง, ทีมเล็ก | องค์กรใหญ่ | Cloud-native || **ข้อดี** | Simple, เร็ว | Queue management, แชร์ resource | Auto-scaling, Isolate || **ตัวอย่าง** | Spark Standalone | AWS EMR, CDH | GKE, EKS |> 💡 **ในคอร์สนี้** เราใช้ `local[*]` ซึ่งหมายถึง "รันบนเครื่องเดียว ใช้ทุก CPU core" — เหมาะสำหรับเรียนรู้

In [ ]:
# ดูว่าเรากำลังใช้ Cluster Manager แบบไหนprint(f"🏢 Master URL: {spark.sparkContext.master}")print()print("📝 ความหมายของ Master URL:")print("  local     = เครื่องเดียว, 1 thread")print("  local[*]  = เครื่องเดียว, ใช้ทุก core")print("  local[4]  = เครื่องเดียว, 4 threads")print("  yarn      = ใช้ YARN บน Hadoop cluster")print("  k8s://... = ใช้ Kubernetes cluster")print("  spark://  = ใช้ Standalone cluster")

# 5. ⏳ Lazy Evaluation (แนวคิดเชิงลึก)## หลักการสำคัญที่สุดของ Spark> **Spark ไม่ทำงานทันที!** Spark จะ "จดบันทึก" ว่าเราต้องการอะไร แล้วค่อยทำตอนที่จำเป็นจริง ๆ### เปรียบเทียบ| | Pandas (Eager) | Spark (Lazy) ||---|---|---|| **เมื่อเขียนคำสั่ง** | ทำทันที | จดไว้ก่อน (สร้าง Plan) || **ข้อดี** | เห็นผลเลย | ปรับแผนให้เร็วที่สุดก่อนทำ || **ข้อเสีย** | ไม่ optimize ข้ามขั้นตอน | ต้อง trigger ด้วย Action |### Transformation vs Action#### 🔹 Transformations (Lazy — จดไว้ก่อน ยังไม่ทำ)| คำสั่ง | หน้าที่ ||--------|--------|| `select()` | เลือกคอลัมน์ || `filter()` / `where()` | กรองแถว || `groupBy()` | จัดกลุ่ม || `join()` | เชื่อมตาราง || `withColumn()` | เพิ่ม/แก้คอลัมน์ || `orderBy()` | เรียงลำดับ |#### 🔸 Actions (ตัวกระตุ้น — สั่งให้ทำจริง!)| คำสั่ง | หน้าที่ ||--------|--------|| `show()` | แสดงข้อมูล || `count()` | นับจำนวนแถว || `collect()` | ดึงข้อมูลทั้งหมดมาที่ Driver || `write()` | เขียนข้อมูลลง disk || `take(n)` | ดึง n แถวแรก |

<div align="center">  <img src="images/lazy_evaluation.png" width="800" alt="Lazy Evaluation: จดออร์เดอร์ (Transformation) vs เสิร์ฟอาหาร (Action)">  <br><i>Lazy Evaluation: จดออร์เดอร์ (Transformation) vs เสิร์ฟอาหาร (Action)</i></div>

### 🔬 Lab: พิสูจน์ Lazy Evaluation

In [ ]:
# สร้างข้อมูลตัวอย่างdata = [(i, f"name_{i}", i * 100, "A" if i % 2 == 0 else "B")        for i in range(1, 10001)]df = spark.createDataFrame(data, ["id", "name", "salary", "dept"])print(f"✅ สร้าง DataFrame ขนาด {df.count()} แถว")df.show(5)

In [ ]:
import time# ===== Transformation (Lazy) — ไม่มีอะไรเกิดขึ้น! =====print("🔹 เริ่มเขียน Transformation...")t0 = time.time()df2 = df.select("id", "salary", "dept")      # เลือกคอลัมน์df3 = df2.filter(df2["salary"] > 500)         # กรองdf4 = df3.groupBy("dept").count()             # จัดกลุ่มt1 = time.time()print(f"⏱️ เวลาที่ใช้ (Transformation): {t1-t0:.4f} วินาที")print("💡 สังเกต: เร็วมาก เพราะ Spark ยังไม่ได้ทำอะไรเลย!")# ===== Action (Trigger!) — ตอนนี้ Spark ทำงานจริง! =====print("\n🔸 เรียก Action (.show()) — Spark เริ่มทำงาน!")t2 = time.time()df4.show()t3 = time.time()print(f"⏱️ เวลาที่ใช้ (Action): {t3-t2:.4f} วินาที")print("💡 สังเกต: ช้ากว่า เพราะ Spark ต้องทำ Transformation ทั้งหมดตอนนี้!")

### ✏️ แบบฝึกหัดที่ 2: Transformation vs Action**โจทย์:** จากโค้ดด้านล่าง ระบุว่าบรรทัดไหนเป็น Transformation (T) หรือ Action (A)

In [ ]:
# แบบฝึกหัดที่ 2: เติม T (Transformation) หรือ A (Action)# บรรทัดที่ 1: df.select("name", "salary")      →  ________# บรรทัดที่ 2: df.filter(df["salary"] > 1000)    →  ________# บรรทัดที่ 3: df.count()                        →  ________# บรรทัดที่ 4: df.groupBy("dept").avg("salary")  →  ________# บรรทัดที่ 5: df.show(10)                       →  ________# บรรทัดที่ 6: df.orderBy("salary")              →  ________# บรรทัดที่ 7: df.collect()                      →  ________# บรรทัดที่ 8: df.withColumn("bonus", df["salary"] * 0.1)  →  ________answers = {    1: "________",    2: "________",    3: "________",    4: "________",    5: "________",    6: "________",    7: "________",    8: "________",}for k, v in answers.items():    print(f"บรรทัดที่ {k}: {v}")

# 6. 📊 DAG (Directed Acyclic Graph)## DAG คืออะไร?DAG คือ **กราฟ** แสดงลำดับการทำงานของ Spark โดย:- **Directed** = มีทิศทาง (ทำจากซ้ายไปขวา)- **Acyclic** = ไม่วนซ้ำ (ไม่กลับไปทำซ้ำ)- **Graph** = แผนภาพเชื่อมโยง```Read CSV → Filter → Select → GroupBy → [Shuffle] → Aggregate → Show  Stage 1 ──────────────────────▶  Stage 2 ────────────────▶```## ขั้นตอนการสร้างแผน1. **Logical Plan** — แปลงจากโค้ดที่เราเขียน2. **Optimized Logical Plan** — Catalyst Optimizer ปรับปรุงให้ดีขึ้น3. **Physical Plan** — แผนจริงที่จะทำ (เลือกอัลกอริทึมที่เร็วที่สุด)> 💡 เปรียบเทียบ: เหมือน GPS — เราบอกจุดหมาย (Logical) → GPS คำนวณเส้นทางที่ดีที่สุด (Optimized) → แสดงเส้นทางจริง (Physical)

In [ ]:
# ดู Execution Plan ของ Sparkdf_plan = df.select("id", "salary", "dept") \            .filter(df["salary"] > 500) \            .groupBy("dept") \            .avg("salary")print("📋 Execution Plan:")print("=" * 50)df_plan.explain("formatted")

In [ ]:
# เปรียบเทียบ: ดูแผนแบบละเอียด (extended)print("📋 Extended Plan (ดู Logical → Physical):")print("=" * 50)df_plan.explain("extended")

# 7. 🔀 Shuffle — การส่งข้อมูลข้ามเครื่อง## Shuffle คืออะไร?Shuffle คือการ **ส่งข้อมูลข้ามเครื่อง** (หรือข้าม partition) เมื่อ Spark ต้องจัดกลุ่มข้อมูลใหม่### เมื่อไหร่เกิด Shuffle?| คำสั่ง | ทำไมต้อง Shuffle | ตัวอย่าง ||--------|-----------------|---------|| `groupBy()` | ต้องรวมข้อมูลกลุ่มเดียวกันมาไว้ที่เดียว | นับยอดขายต่อจังหวัด || `join()` | ต้องจับคู่ข้อมูลจาก 2 ตาราง | JOIN orders กับ customers || `orderBy()` | ต้องเรียงข้อมูลทั้งหมด | เรียงตามราคาจากมากไปน้อย || `distinct()` | ต้องเช็คค่าซ้ำทั้งหมด | หาจำนวน user ไม่ซ้ำ |### ทำไม Shuffle ถึง "แพง"?> ⚠️ **Shuffle = การส่งข้อมูลผ่านเครือข่าย = ช้า!**```เครื่อง 1: [A1, B2, A3]  ──── Network ────▶  เครื่อง 1: [A1, A3, A5] (กลุ่ม A)เครื่อง 2: [B4, A5, B6]  ──── Network ────▶  เครื่อง 2: [B2, B4, B6] (กลุ่ม B)```- ต้องเขียน disk ก่อนส่ง (Spill)- ส่งผ่าน Network (ช้ากว่า RAM 1000 เท่า)- เครื่องรับต้องรอข้อมูลครบก่อนทำงานต่อ

<div align="center">  <img src="images/shuffle_concept.png" width="800" alt="Shuffle Concept: การส่งข้อมูลข้ามเครื่องเหมือนการแยกสีลูกบอลลงถังที่ถูกต้อง">  <br><i>Shuffle Concept: การส่งข้อมูลข้ามเครื่องเหมือนการแยกสีลูกบอลลงถังที่ถูกต้อง</i></div>

In [ ]:
# สาธิตความแตกต่าง: คำสั่งที่มี Shuffle vs ไม่มี Shuffleimport time# สร้างข้อมูลขนาดใหญ่ขึ้นbig_data = [(i, f"dept_{i % 50}", i * 10.5) for i in range(100000)]big_df = spark.createDataFrame(big_data, ["id", "department", "amount"])# ===== ไม่มี Shuffle (Narrow Transformation) =====t0 = time.time()result1 = big_df.filter(big_df["amount"] > 5000).select("id", "amount")result1.count()  # trigger actiont1 = time.time()print(f"🟢 Filter + Select (ไม่มี Shuffle): {t1-t0:.3f} วินาที")# ===== มี Shuffle (Wide Transformation) =====t2 = time.time()result2 = big_df.groupBy("department").sum("amount")result2.count()  # trigger actiont3 = time.time()print(f"🔴 GroupBy + Sum (มี Shuffle): {t3-t2:.3f} วินาที")print(f"\n💡 สังเกต: GroupBy ช้ากว่า Filter เพราะต้อง Shuffle!")

# 8. 🧪 Lab: โหลดและวิเคราะห์ข้อมูลจริงด้วย Spark### Step 1: โหลดข้อมูล CSV

In [ ]:
# ใช้ข้อมูลตัวอย่างที่สร้างขึ้น (สำหรับ Colab ที่ไม่มี crime.csv)import randomrandom.seed(42)crime_types = ["THEFT", "BATTERY", "ASSAULT", "BURGLARY", "NARCOTICS",               "ROBBERY", "MOTOR VEHICLE THEFT", "CRIMINAL DAMAGE"]locations = ["STREET", "APARTMENT", "RESIDENCE", "SIDEWALK", "PARKING LOT"]crime_data = [    (i, random.choice(crime_types), random.choice(locations),     random.randint(2018, 2023), random.choice([True, False]))    for i in range(1, 50001)]df_crime = spark.createDataFrame(    crime_data,    ["case_id", "primary_type", "location", "year", "arrest"])print(f"✅ โหลดข้อมูลสำเร็จ: {df_crime.count()} แถว")print("\n📊 Schema:")df_crime.printSchema()print("\n📋 ตัวอย่างข้อมูล 5 แถวแรก:")df_crime.show(5)

### Step 2: Transformation (Lazy) — ยังไม่เกิดอะไร

In [ ]:
# Transformation: เลือกเฉพาะ primary_type ที่ไม่ใช่ nulldf_filtered = df_crime.select("primary_type", "year", "arrest") \                      .filter(df_crime["primary_type"].isNotNull())print("✅ สร้าง Transformation เสร็จ (แต่ Spark ยังไม่ทำงาน!)")print(f"Type ของ df_filtered: {type(df_filtered)}")

### Step 3: Action — สั่งให้ Spark ทำงาน

In [ ]:
# Action: นับจำนวนแถว (Spark เริ่มทำงานจริง!)count_result = df_filtered.count()print(f"📊 จำนวนอาชญากรรมที่กรองแล้ว: {count_result:,} แถว")

### Step 4: ดู Execution Plan

In [ ]:
# ดู Plan ว่า Spark จะทำงานยังไงprint("📋 Execution Plan:")df_filtered.explain("formatted")

# 9. 💾 Cache — เก็บข้อมูลใน RAM เพื่อใช้ซ้ำ## เมื่อไหร่ควรใช้ Cache?ใช้ `cache()` เมื่อ **DataFrame ถูกใช้ซ้ำหลายครั้ง**```ไม่ cache: Read → Filter → Count (อ่านใหม่ทุกครั้ง!)           Read → Filter → Show  (อ่านใหม่อีก!)           cache:     Read → Filter → [Cache ใน RAM]                              ├─→ Count (อ่านจาก RAM!)                              └─→ Show  (อ่านจาก RAM!)```

In [ ]:
import time# สร้าง DataFrame ที่ใช้ซ้ำdf_analysis = df_crime.filter(df_crime["year"] >= 2020)# ===== ไม่ใช้ Cache =====print("🔴 ไม่ใช้ Cache:")t0 = time.time()df_analysis.count()df_analysis.groupBy("primary_type").count().show(5, truncate=False)t1 = time.time()print(f"⏱️ เวลา: {t1-t0:.3f} วินาที\n")# ===== ใช้ Cache =====print("🟢 ใช้ Cache:")df_analysis.cache()  # บอก Spark ให้เก็บใน RAMt2 = time.time()df_analysis.count()  # ครั้งแรก: อ่านจาก source + เก็บ cachedf_analysis.groupBy("primary_type").count().show(5, truncate=False)t3 = time.time()print(f"⏱️ เวลา: {t3-t2:.3f} วินาที")# อย่าลืม unpersist เมื่อไม่ใช้แล้ว!df_analysis.unpersist()print("\n🗑️ Unpersist cache เรียบร้อย (คืน RAM)")

# 10. ⚠️ ข้อผิดพลาดที่พบบ่อย (Common Pitfalls)| ❌ สิ่งที่ไม่ควรทำ | 💥 ผลเสีย | ✅ ทำแบบนี้แทน ||---|---|---|| `df.collect()` บนข้อมูลใหญ่ | Driver crash (หน่วยความจำเต็ม) | ใช้ `df.show()` หรือ `df.take(10)` || JOIN ตารางใหญ่ 2 ตาราง โดยไม่ filter ก่อน | Shuffle ข้อมูลมหาศาล | Filter ก่อน JOIN เสมอ || `inferSchema=True` บน CSV ใหญ่ | Spark ต้องอ่านทั้งไฟล์เพื่อเดา type | กำหนด Schema เอง || Partition มากเกินไป (เช่น 10,000) | Overhead จาก scheduling | ใช้ `coalesce()` ลด partition || Partition น้อยเกินไป (เช่น 1) | ไม่ได้ประโยชน์จาก parallelism | ใช้ `repartition()` เพิ่ม |

In [ ]:
# ตัวอย่าง: กำหนด Schema เอง (เร็วกว่า inferSchema)from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType# ❌ แบบช้า (inferSchema)# df = spark.read.csv("big_file.csv", header=True, inferSchema=True)# ✅ แบบเร็ว (กำหนด Schema เอง)schema = StructType([    StructField("case_id", IntegerType(), False),    StructField("primary_type", StringType(), True),    StructField("location", StringType(), True),    StructField("year", IntegerType(), True),    StructField("arrest", BooleanType(), True),])print("✅ Schema ที่กำหนดเอง:")print(schema)print("\n💡 การกำหนด Schema เอง ช่วยให้ Spark ไม่ต้องสแกนไฟล์ทั้งหมดเพื่อเดา type")

# 11. 📝 แบบฝึกหัดรวม (Comprehensive Exercises)### แบบฝึกหัดที่ 3: วิเคราะห์ข้อมูลอาชญากรรมด้วย Spark**โจทย์:** ใช้ `df_crime` ที่โหลดไว้แล้ว ทำตามขั้นตอนด้านล่าง

In [ ]:
# แบบฝึกหัดที่ 3.1: นับจำนวนอาชญากรรมแต่ละประเภท (primary_type)# เรียงจากมากไปน้อย แสดง 5 อันดับแรก# TODO: เติมโค้ดresult_3_1 = df_crime.groupBy(________) \                     .count() \                     .orderBy(________) \                     .limit(5)result_3_1.show()

In [ ]:
# แบบฝึกหัดที่ 3.2: หาอัตราการจับกุม (arrest rate) ต่อประเภทอาชญากรรม# Hint: ใช้ avg("arrest") เพราะ True=1, False=0# TODO: เติมโค้ดfrom pyspark.sql import functions as Fresult_3_2 = df_crime.groupBy(________) \                     .agg(                         F.count("*").alias("total_cases"),                         F.________(________).alias("arrest_rate")                     ) \                     .orderBy(F.col("arrest_rate").desc())result_3_2.show(truncate=False)

In [ ]:
# แบบฝึกหัดที่ 3.3: วิเคราะห์แนวโน้มรายปี# หาจำนวนอาชญากรรมต่อปี เรียงตามปี# TODO: เติมโค้ดresult_3_3 = df_crime.groupBy(________) \                     .________ \                     .orderBy(________)result_3_3.show()

### แบบฝึกหัดที่ 4: ดู Execution Plan**โจทย์:** เขียน query ที่มี Shuffle และดู plan

In [ ]:
# แบบฝึกหัดที่ 4: สร้าง query แล้วดู plan# TODO: เขียน query ที่ทำ 3 อย่าง:# 1) Filter เฉพาะปี 2020 ขึ้นไป# 2) GroupBy ตาม primary_type# 3) นับจำนวน + เรียงจากมากไปน้อยmy_query = df_crime \    .________(________) \    .________(________) \    .________ \    .orderBy(F.col("count").desc())# ดู plan (ไม่ต้องแก้ส่วนนี้)print("📋 Execution Plan ของ query คุณ:")my_query.explain("formatted")print("\n📊 ผลลัพธ์:")my_query.show()

### แบบฝึกหัดที่ 5: Cache Performance**โจทย์:** พิสูจน์ว่า Cache ช่วยให้เร็วขึ้นจริง

In [ ]:
# แบบฝึกหัดที่ 5: เปรียบเทียบ Cache vs ไม่ Cacheimport time# สร้าง DataFrame ที่ซับซ้อนdf_complex = df_crime.filter(df_crime["year"] >= 2020) \                     .withColumn("case_category",                         F.when(F.col("arrest") == True, "ARRESTED")                          .otherwise("NOT_ARRESTED"))# TODO: เติมโค้ด# 1) วัดเวลา ไม่ใช้ cache (รัน count + groupBy 2 ครั้ง)t0 = time.time()count1 = df_complex.________  # นับจำนวนgroup1 = df_complex.groupBy("case_category").count().________  # collect ผลt1 = time.time()no_cache_time = t1 - t0# 2) Cache แล้ววัดเวลาใหม่df_complex.________  # cachet2 = time.time()count2 = df_complex.________  # นับจำนวนgroup2 = df_complex.groupBy("case_category").count().________  # collect ผลt3 = time.time()with_cache_time = t3 - t2print(f"🔴 ไม่ใช้ Cache: {no_cache_time:.3f} วินาที")print(f"🟢 ใช้ Cache: {with_cache_time:.3f} วินาที")# อย่าลืม unpersist!df_complex.unpersist()

# 12. 📌 สรุป (Summary)## สิ่งที่เรียนรู้วันนี้| หัวข้อ | สรุป ||--------|------|| **เครื่องเดียวไม่พอ** | เมื่อข้อมูล > RAM/CPU → ต้องใช้ Distributed Computing || **Spark คืออะไร** | Distributed Compute Engine (ไม่ใช่ DB, ไม่ใช่ Storage) || **Architecture** | Driver (สมอง) + Executors (แรงงาน) + Cluster Manager (ผู้จัดสรร) || **Lazy Evaluation** | Transformation สร้างแผน, Action สั่งทำ → ช่วย optimize || **DAG** | กราฟแสดงลำดับการทำงาน → Catalyst Optimizer ปรับให้เร็ว || **Shuffle** | ส่งข้อมูลข้ามเครื่อง → ช้า แพง ต้องระวัง || **Cache** | เก็บ DataFrame ใน RAM สำหรับใช้ซ้ำ → ประหยัดเวลา || **explain()** | ดูแผนการทำงาน → เข้าใจว่า Spark จะทำอะไร |## 🔮 สัปดาห์หน้า: Data Pipeline> สัปดาห์ที่ 5: กระบวนการจัดการข้อมูล (Data Pipeline)  > ingest → clean → transform → store → analyze---## 📚 แหล่งเรียนรู้เพิ่มเติม- [Apache Spark Documentation](https://spark.apache.org/docs/latest/)- [Spark: The Definitive Guide](https://www.oreilly.com/library/view/spark-the-definitive/9781491912201/)- [PySpark API Reference](https://spark.apache.org/docs/latest/api/python/)

In [ ]:
# ปิด Spark Sessionspark.stop()print("✅ คาบวันนี้จบแล้ว! สัปดาห์หน้า: Data Pipeline 🚀")